In [1]:
import android_manager
import poke_manager
import win32gui
import win32con
import pyautogui
import time
from ppadb.client import Client as AdbClient
from ppadb.device import Device as AdbDevice
import importlib
import os
from pathlib import Path

In [2]:
importlib.reload(android_manager)
importlib.reload(poke_manager)

#scrcpy로 Device가 실행된 상태
cur_client = AdbClient(host='localhost', port=5037)
#serial number:
#ce10171ab2312a0d04 for s8
#ce071717d4035622047e for note8
pm_s8 = poke_manager.Poke_device(cur_client.device('ce10171ab2312a0d04'), 2)
pm_note8 = poke_manager.Poke_device(cur_client.device('ce071717d4035622047e'), 1)
#my-pm은 둘 중 하나를 복사
my_pm = pm_s8
#my_pm = pm_note8
#always-on-top 옵션 주기 > 나중에는 클래스 안에서 구현. 
pm_devices = [d for d in [pm_s8, pm_note8] if d is not None]

(3110, 1335)
Deviced SM-G950N is connedted
Hwnd: 526696
(3470, 1335)
Deviced SM-N950N is connedted
Hwnd: 393758


In [6]:
my_pm.buy_100(20)

100골 1개 구매함. 남은 선물: 20
100골 2개 구매함. 남은 선물: 19
100골 3개 구매함. 남은 선물: 18
100골 4개 구매함. 남은 선물: 17
100골 5개 구매함. 남은 선물: 16
100골 6개 구매함. 남은 선물: 15
100골 7개 구매함. 남은 선물: 14
100골 8개 구매함. 남은 선물: 13
100골 9개 구매함. 남은 선물: 12
100골 10개 구매함. 남은 선물: 11
100골 11개 구매함. 남은 선물: 10
100골 12개 구매함. 남은 선물: 9
100골 13개 구매함. 남은 선물: 8
100골 14개 구매함. 남은 선물: 7
100골 15개 구매함. 남은 선물: 6
100골 16개 구매함. 남은 선물: 5
100골 17개 구매함. 남은 선물: 4
100골 18개 구매함. 남은 선물: 3
100골 19개 구매함. 남은 선물: 2
100골 20개 구매함. 남은 선물: 1


In [13]:
#선물열기 자동화
my_pm.confidence = 0.95
my_pm.status = 'initial_screen'
profile_xy = (137, 1892)
profile_friend_xy = (755, 163)
profile_sort_xy = (934, 1921)
profile_sort_present_xy = (927, 1534)

while True:
    my_pm.prev_status = my_pm.status
    my_pm.automation_by_image('initial_screen', profile_xy, new_status='profile')
    my_pm.automation_by_image('profile_me', profile_friend_xy, cur_status='profile', new_status='profile_friend')
    my_pm.automation_by_image('profile_friend', no_click=True, cur_status='profile', new_status='profile_friend')
    if my_pm.status == 'profile_friend' and not my_pm.automation_by_image('present', cur_status='profile_friend', new_status='open_present'):
        print("오픈할 선물이 없음")
        #선물로 소팅되어 있는 경우 나갔다가 들어옴
        if my_pm.automation_by_image('profile_friend_sort_present_desc', target_img='profile_friend_x_green', new_status = 'initial'):
            pass
        else:
            #선물로 소팅하기 
            my_pm.click_xy(profile_sort_xy, is_device_xy=True)
            my_pm.click_xy(profile_sort_present_xy, is_device_xy=True)
            time.sleep(2)
            my_pm.automation_by_image('sort_present_asc')
    if my_pm.automation_by_image('open_present_complete', target_img='x_white', cur_status='after_open_present'):
        my_pm.status = 'finished_open_present'
        break
    my_pm.automation_by_image('present_to_open', cur_status='open_present', confidence=0.7)
    my_pm.automation_by_image('open_cmd', new_status='profile_friend_detail')
    my_pm.automation_by_image('exchange_new_record', target_img='x_white', cur_status='profile_friend_detail')
    my_pm.automation_by_image('profile_friend_detail_x_green', cur_status='profile_friend_detail', new_status='profile_friend')
    my_pm.automation_by_image('initial_seemore', target_img='x_green', new_status='initial')
    if my_pm.prev_status == my_pm.status:
        print(f'Status: {my_pm.status}')
        set_status_by_image()

Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Found initial_screen at Point(x=3294, y=2007)
In automation_by_img(), 
Img: initial_screen, T_img: None
Click: (137, 1892)(is_tap_xy: True)
Tapping xy: (137, 1892)
setting new status:profile, prev status: profile
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: initial_screen, prev_status: initial_screen
Status: initial_screen
Setting Status: i

KeyboardInterrupt: 

In [12]:
#화면의 이미지를 통해서 현재 상태를 리턴해주는 함수
def set_status_by_image():
    my_pm.prev_status = my_pm.status
    my_pm.automation_by_image('initial_screen', no_click=True, new_status='initial_screen')
    my_pm.automation_by_image('profile_me', no_click=True, new_status='profile_me')
    my_pm.automation_by_image('profile_friend', no_click=True, new_status='profile_friend')
    my_pm.automation_by_image('present_to_open', no_click=True, new_status='open_present')
    my_pm.automation_by_image('profile_friend_detail', no_click=True, new_status='profile_friend_detail')
    print(f'Setting Status: {my_pm.status}, prev_status: {my_pm.prev_status}')

In [4]:
#한영 자동전환
keyboard_language_change_xy = (329,1966)
#수원
text_suwon = 'tndnjs&vkxmsj vhzptahs0&!wjstjfdml vhzptahs'
#서울
text_seoul = 'tjdnf&vkxmsj vhzptahs0&!wjstjfdml vhzptahs'
#한글로 전환
my_pm.automation_by_image('keyboard_qwerty', device_xy=keyboard_language_change_xy)
my_pm.device.input_text(text_seoul)
my_pm.automation_by_image('keyboard_confirm')
#
pm_note8.automation_by_image('keyboard_qwerty', device_xy=keyboard_language_change_xy)
pm_note8.device.input_text(text_suwon)
pm_note8.automation_by_image('keyboard_confirm')

Found keyboard_confirm at Point(x=3439, y=2027)
In automation_by_img(), 
Img: keyboard_confirm, T_img: None
Click: Point(x=3439, y=2027)(is_tap_xy: False)
Tapping xy: (972, 1926)


False

In [8]:
#교환 자동화
first_pokemon_in_list = (191, 734)
exchange_special_change_no = (523, 1302)
my_pm.confidence = 0.9

while True:
    my_pm.automation_by_image('profile_friend_detail_exchange', new_status='before_exchange')
    my_pm.automation_by_image('exchange_pokemon_list', device_xy=first_pokemon_in_list)
    my_pm.automation_by_image('exchange_next_cmd')
    my_pm.automation_by_image('exchange_100_cost', target_img='exchange_confirm_cmd', new_status='after_exchange')
    my_pm.automation_by_image('exchange_special_change', exchange_special_change_no)
    my_pm.automation_by_image('x_green', cur_status='after_exchange', new_status='before_exchange')

Found profile_friend_detail_exchange at Point(x=3406, y=1916)
In automation_by_img(), 
Img: profile_friend_detail_exchange, T_img: None
Click: Point(x=3406, y=1916)(is_tap_xy: False)
Tapping xy: (873, 1593)
setting new status:before_exchange, prev status: before_exchange
Found exchange_pokemon_list at Point(x=3265, y=1473)
In automation_by_img(), 
Img: exchange_pokemon_list, T_img: None
Click: (191, 734)(is_tap_xy: True)
Tapping xy: (191, 734)
Found exchange_next_cmd at Point(x=3292, y=1940)
In automation_by_img(), 
Img: exchange_next_cmd, T_img: None
Click: Point(x=3292, y=1940)(is_tap_xy: False)
Tapping xy: (531, 1665)
Found exchange_100_cost at Point(x=3173, y=1738)
In automation_by_img(), 
Img: exchange_100_cost, T_img: exchange_confirm_cmd
Click: Point(x=3148, y=1737)(is_tap_xy: False)
Tapping xy: (99, 1056)
setting new status:after_exchange, prev status: before_exchange
Found exchange_100_cost at Point(x=3225, y=1738)
Found exchange_100_cost at Point(x=3225, y=1738)
Found exchang

KeyboardInterrupt: 